In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import math
import sklearn
import sklearn.datasets
import sys
sys.path.append('/content/drive/MyDrive/Deep_Learning_TA/Optimizations/')
from utils import load_params_and_grads, initialize_parameters, forward_propagation, backward_propagation
from utils import compute_cost, predict, predict_dec, plot_decision_boundary, load_dataset

ModuleNotFoundError: ignored

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (7.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
train_X, train_Y = load_dataset()
print(train_X.shape)

In [ ]:
def update_parameters_with_gd(parameters, grads, learning_rate):
    """
    Update parameters using one step of gradient descent

    Arguments:
    parameters -- python dictionary containing your parameters to be updated:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing your gradients to update each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    learning_rate -- the learning rate, scalar.

    Returns:
    parameters -- python dictionary containing your updated parameters
    """

    L = len(parameters) // 2 # number of layers in the neural networks

    # Update rule for each parameter
    for l in range(L):

        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate*grads['dW' + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*grads['db' + str(l+1)]

    return parameters

In [ ]:
def initialize_velocity(parameters):
    """
    Initializes the velocity as a python dictionary with:
                - keys: "dW1", "db1", ..., "dWL", "dbL"
                - values: numpy arrays of zeros of the same shape as the corresponding gradients/parameters.
    Arguments:
    parameters -- python dictionary containing your parameters.
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl

    Returns:
    v -- python dictionary containing the current velocity.
                    v['dW' + str(l)] = velocity of dWl
                    v['db' + str(l)] = velocity of dbl
    """

    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}

    # Initialize velocity
    for l in range(L):

        v["dW" + str(l+1)] = np.zeros((parameters["W" + str(l+1)].shape[0], parameters["W" + str(l+1)].shape[1]))
        v["db" + str(l+1)] = np.zeros((parameters["b" + str(l+1)].shape[0], parameters["b" + str(l+1)].shape[1]))

    return v

In [ ]:
def initialize_adam(parameters) :
    """
    Initializes v and s as two python dictionaries with:
                - keys: "dW1", "db1", ..., "dWL", "dbL"
                - values: numpy arrays of zeros of the same shape as the corresponding gradients/parameters.

    Arguments:
    parameters -- python dictionary containing your parameters.
                    parameters["W" + str(l)] = Wl
                    parameters["b" + str(l)] = bl

    Returns:
    v -- python dictionary that will contain the exponentially weighted average of the gradient.
                    v["dW" + str(l)] = ...
                    v["db" + str(l)] = ...
    s -- python dictionary that will contain the exponentially weighted average of the squared gradient.
                    s["dW" + str(l)] = ...
                    s["db" + str(l)] = ...

    """

    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}
    s = {}

    # Initialize v, s. Input: "parameters". Outputs: "v, s".
    for l in range(L):

        v["dW" + str(l+1)] = np.zeros((parameters["W" + str(l+1)].shape[0], parameters["W" + str(l+1)].shape[1]))
        v["db" + str(l+1)] = np.zeros((parameters["b" + str(l+1)].shape[0], parameters["b" + str(l+1)].shape[1]))
        s["dW" + str(l+1)] = np.zeros((parameters["W" + str(l+1)].shape[0], parameters["W" + str(l+1)].shape[1]))
        s["db" + str(l+1)] = np.zeros((parameters["b" + str(l+1)].shape[0], parameters["b" + str(l+1)].shape[1]))

    return v, s

In [ ]:
def update_parameters_with_momentum(parameters, grads, v, beta, learning_rate):
    """
    Update parameters using Momentum

    Arguments:
    parameters -- python dictionary containing your parameters:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing your gradients for each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    v -- python dictionary containing the current velocity:
                    v['dW' + str(l)] = ...
                    v['db' + str(l)] = ...
    beta -- the momentum hyperparameter, scalar
    learning_rate -- the learning rate, scalar

    Returns:
    parameters -- python dictionary containing your updated parameters
    v -- python dictionary containing your updated velocities
    """

    L = len(parameters) // 2 # number of layers in the neural networks

    # Momentum update for each parameter
    for l in range(L):

        # compute velocities
        v["dW" + str(l+1)] = beta*v["dW" + str(l+1)] + (1 - beta)*grads['dW' + str(l+1)]
        v["db" + str(l+1)] = beta*v["db" + str(l+1)] + (1 - beta)*grads['db' + str(l+1)]
        # update parameters
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate*v["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*v["db" + str(l+1)]

    return parameters, v

In [ ]:
def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate = 0.01,
                                beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):
    """
    Update parameters using Adam

    Arguments:
    parameters -- python dictionary containing your parameters:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing your gradients for each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    learning_rate -- the learning rate, scalar.
    beta1 -- Exponential decay hyperparameter for the first moment estimates
    beta2 -- Exponential decay hyperparameter for the second moment estimates
    epsilon -- hyperparameter preventing division by zero in Adam updates

    Returns:
    parameters -- python dictionary containing your updated parameters
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    """

    L = len(parameters) // 2                 # number of layers in the neural networks
    v_corrected = {}                         # Initializing first moment estimate, python dictionary
    s_corrected = {}                         # Initializing second moment estimate, python dictionary

    # Perform Adam update on all parameters
    for l in range(L):
        # Moving average of the gradients. Inputs: "v, grads, beta1". Output: "v".

        v["dW" + str(l+1)] = beta1*v["dW" + str(l+1)] + (1 - beta1)*grads['dW' + str(l+1)]
        v["db" + str(l+1)] = beta1*v["db" + str(l+1)] + (1 - beta1)*grads['db' + str(l+1)]

        # Compute bias-corrected first moment estimate. Inputs: "v, beta1, t". Output: "v_corrected".

        v_corrected["dW" + str(l+1)] = v["dW" + str(l+1)]/(1 - beta1**t)
        v_corrected["db" + str(l+1)] = v["db" + str(l+1)]/(1 - beta1**t)


        # Moving average of the squared gradients. Inputs: "s, grads, beta2". Output: "s".

        s["dW" + str(l+1)] = beta2*s["dW" + str(l+1)] + (1 - beta2)*np.square(grads['dW' + str(l+1)])
        s["db" + str(l+1)] = beta2*s["db" + str(l+1)] + (1 - beta2)*np.square(grads['db' + str(l+1)])


        # Compute bias-corrected second raw moment estimate. Inputs: "s, beta2, t". Output: "s_corrected".

        s_corrected["dW" + str(l+1)] = s["dW" + str(l+1)]/(1 - beta2**t)
        s_corrected["db" + str(l+1)] = s["db" + str(l+1)]/(1 - beta2**t)

        # Update parameters. Inputs: "parameters, learning_rate, v_corrected, s_corrected, epsilon". Output: "parameters".

        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate*v_corrected["dW" + str(l+1)]/(np.sqrt(s_corrected["dW" + str(l+1)])+epsilon)
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*v_corrected["db" + str(l+1)]/(np.sqrt(s_corrected["db" + str(l+1)])+epsilon)


    return parameters, v, s

In [ ]:
def initialize_adagrad(parameters):
    """
    Initializes the variables for AdaGrad optimizer.

    Arguments:
    parameters -- python dictionary containing the parameters to be optimized.

    Returns:
    v -- python dictionary containing the accumulated squared gradient for each parameter.
    """
    L = len(parameters) // 2
    v = {}

    for l in range(L):
        v["dW" + str(l + 1)] = np.zeros_like(parameters["W" + str(l + 1)])
        v["db" + str(l + 1)] = np.zeros_like(parameters["b" + str(l + 1)])

    return v


In [ ]:
def update_parameters_with_adagrad(parameters, grads, v, learning_rate, epsilon=1e-8):
    """
    Update parameters using AdaGrad optimizer.

    Arguments:
    parameters -- python dictionary containing the parameters to be optimized.
    grads -- python dictionary containing the gradients of the cost function.
    v -- python dictionary containing the accumulated squared gradient for each parameter.
    learning_rate -- the learning rate, scalar.
    epsilon -- hyperparameter preventing division by zero.

    Returns:
    parameters -- python dictionary containing your updated parameters.
    """
    L = len(parameters) // 2

    for l in range(L):
        v["dW" + str(l + 1)] += grads["dW" + str(l + 1)] ** 2
        v["db" + str(l + 1)] += grads["db" + str(l + 1)] ** 2

        parameters["W" + str(l + 1)] -= (learning_rate / (np.sqrt(v["dW" + str(l + 1)]) + epsilon)) * grads["dW" + str(l + 1)]
        parameters["b" + str(l + 1)] -= (learning_rate / (np.sqrt(v["db" + str(l + 1)]) + epsilon)) * grads["db" + str(l + 1)]

    return parameters,v

In [ ]:
def initialize_rmsprop(parameters):
    """
    Initializes the variables for RMSprop optimizer.

    Arguments:
    parameters -- python dictionary containing the parameters to be optimized.

    Returns:
    s -- python dictionary containing the moving average of squared gradient for each parameter.
    """
    L = len(parameters) // 2
    s = {}

    for l in range(L):
        s["dW" + str(l + 1)] = np.zeros_like(parameters["W" + str(l + 1)])
        s["db" + str(l + 1)] = np.zeros_like(parameters["b" + str(l + 1)])

    return s

In [ ]:
def update_parameters_with_rmsprop(parameters, grads, s, learning_rate, beta=0.9, epsilon=1e-8):
    """
    Update parameters using RMSprop optimizer.

    Arguments:
    parameters -- python dictionary containing the parameters to be optimized.
    grads -- python dictionary containing the gradients of the cost function.
    s -- python dictionary containing the moving average of squared gradient for each parameter.
    learning_rate -- the learning rate, scalar.
    beta -- the decay rate for the moving average (usually 0.9).
    epsilon -- hyperparameter preventing division by zero.

    Returns:
    parameters -- python dictionary containing your updated parameters.
    """
    L = len(parameters) // 2

    for l in range(L):
        s["dW" + str(l + 1)] = beta * s["dW" + str(l + 1)] + (1 - beta) * (grads["dW" + str(l + 1)] ** 2)
        s["db" + str(l + 1)] = beta * s["db" + str(l + 1)] + (1 - beta) * (grads["db" + str(l + 1)] ** 2)

        parameters["W" + str(l + 1)] -= (learning_rate / (np.sqrt(s["dW" + str(l + 1)]) + epsilon)) * grads["dW" + str(l + 1)]
        parameters["b" + str(l + 1)] -= (learning_rate / (np.sqrt(s["db" + str(l + 1)]) + epsilon)) * grads["db" + str(l + 1)]

    return parameters,s

In [ ]:
def model(X, Y, layers_dims, optimizer, learning_rate=0.0007, beta=0.9,
          beta1=0.9, beta2=0.999, epsilon=1e-8, num_epochs=10000, print_cost=True):
    """
    3-layer neural network model trained using full batch gradient descent.

    Arguments:
    X -- input data, of shape (n_features, number of examples)
    Y -- true "label" vector (1 for blue dot / 0 for red dot), of shape (1, number of examples)
    layers_dims -- python list, containing the size of each layer
    learning_rate -- the learning rate, scalar.
    beta -- Momentum hyperparameter
    beta1 -- Exponential decay hyperparameter for the past gradients estimates
    beta2 -- Exponential decay hyperparameter for the past squared gradients estimates
    epsilon -- hyperparameter preventing division by zero in Adam updates
    num_epochs -- number of epochs
    print_cost -- True to print the cost every 1000 epochs

    Returns:
    parameters -- python dictionary containing your updated parameters
    """

    L = len(layers_dims)  # number of layers in the neural networks
    costs = []  # to keep track of the cost
    t = 0  # initializing the counter required for Adam update
    m = X.shape[1]  # number of training examples

    # Initialize parameters
    parameters = initialize_parameters(layers_dims)

    # Initialize the optimizer
    if optimizer == "gd":
        pass  # no initialization required for gradient descent
    elif optimizer == "momentum":
        v = initialize_velocity(parameters)
    elif optimizer == "adam":
        v, s = initialize_adam(parameters)
    elif optimizer == "rmsprop":
      s = initialize_rmsprop(parameters)
    elif optimizer == "adagrad":
      v = initialize_adagrad(parameters)


    # Optimization loop
    for i in range(num_epochs):

        # Forward propagation
        a3, caches = forward_propagation(X, parameters)

        # Compute cost
        cost = compute_cost(a3, Y)

        # Backward propagation
        grads = backward_propagation(X, Y, caches)

        # Update parameters
        if optimizer == "gd":
            parameters = update_parameters_with_gd(parameters, grads, learning_rate)
        elif optimizer == "momentum":
            parameters, v = update_parameters_with_momentum(parameters, grads, v, beta, learning_rate)
        elif optimizer == "adam":
            t = t + 1  # Adam counter
            parameters, v, s = update_parameters_with_adam(parameters, grads, v, s,
                                                           t, learning_rate, beta1, beta2, epsilon)
        elif optimizer == "rmsprop":
            parameters,s = update_parameters_with_rmsprop(parameters, grads, s, learning_rate, beta=0.9, epsilon=1e-8)
        elif optimizer == "adagrad":
            parameters,v = update_parameters_with_adagrad(parameters, grads, v, learning_rate, epsilon=1e-8)


        # Print the cost every 1000 epochs
        if print_cost and i % 1000 == 0:
            print("Cost after epoch %i: %f" % (i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)

    # Plot the cost
    plt.plot(costs)
    plt.ylabel('Cost')
    plt.xlabel('Epochs (per 1000)')
    plt.title("Learning rate = " + str(learning_rate))
    plt.show()

    return parameters


In [ ]:
layers_dims = [train_X.shape[0], 10, 4, 1]
parameters = model(train_X, train_Y, layers_dims,"gd",0.005)

# Predict
predictions = predict(train_X, train_Y, parameters)

# Plot decision boundary
plt.title("Model with Gradient Descent optimization")
axes = plt.gca()
axes.set_xlim([-1.5,2.5])
axes.set_ylim([-1,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

In [ ]:
layers_dims = [train_X.shape[0], 10, 4, 1]
parameters = model(train_X, train_Y, layers_dims,"momentum",0.005,0.99)

# Predict
predictions = predict(train_X, train_Y, parameters)

# Plot decision boundary
plt.title("Model with Momentum optimization")
axes = plt.gca()
axes.set_xlim([-1.5,2.5])
axes.set_ylim([-1,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

In [ ]:
# train 3-layer model
layers_dims = [train_X.shape[0], 10, 4, 1]
parameters = model(train_X, train_Y, layers_dims,"adam",0.005)

# Predict
predictions = predict(train_X, train_Y, parameters)

# Plot decision boundary
plt.title("Model with Adam optimization")
axes = plt.gca()
axes.set_xlim([-1.5,2.5])
axes.set_ylim([-1,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

In [ ]:
# train 3-layer model
layers_dims = [train_X.shape[0], 10, 4, 1]
parameters = model(train_X, train_Y, layers_dims,"adagrad",0.005)

# Predict
predictions = predict(train_X, train_Y, parameters)

# Plot decision boundary
plt.title("Model with Adagrad optimization")
axes = plt.gca()
axes.set_xlim([-1.5,2.5])
axes.set_ylim([-1,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

In [ ]:
# train 3-layer model
layers_dims = [train_X.shape[0], 10, 4, 1]
parameters = model(train_X, train_Y, layers_dims,"rmsprop",0.005)

# Predict
predictions = predict(train_X, train_Y, parameters)

# Plot decision boundary
plt.title("Model with RMSProp optimization")
axes = plt.gca()
axes.set_xlim([-1.5,2.5])
axes.set_ylim([-1,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

In [ ]:
def initialize_parameters_with_batch_norm(layers_dims):
    """
    Initializes parameters for a neural network with batch normalization.

    Arguments:
    layers_dims -- python list, containing the size of each layer.

    Returns:
    parameters -- python dictionary containing the initialized parameters.
    """
    L = len(layers_dims) - 1
    parameters = {}
    for l in range(1, L + 1):
        parameters['W' + str(l)] = np.random.randn(layers_dims[l], layers_dims[l - 1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layers_dims[l], 1))
        parameters['gamma' + str(l)] = np.ones((layers_dims[l], 1))
        parameters['beta' + str(l)] = np.zeros((layers_dims[l], 1))
    return parameters



def compute_cost_with_batch_norm(AL, Y):
    """
    Computes the cost for a neural network with batch normalization.

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples).
    Y -- true "label" vector, shape (1, number of examples).

    Returns:
    cost -- cross-entropy cost.
    """
    m = Y.shape[1]
    cost = (-1 / m) * np.sum(Y * np.log(AL) + (1 - Y) * np.log(1 - AL))
    return cost


In [ ]:
def forward_propagation_with_batch_norm(X, parameters):
    """
    Implements the forward propagation with batch normalization.

    Arguments:
    X -- input data, of shape (n_features, number of examples).
    parameters -- python dictionary containing the parameters.

    Returns:
    A -- output of the last layer.
    caches -- list of caches containing all the intermediate values.
    """
    caches = []
    A = X
    L = len(parameters) // 4

    for l in range(1, L):
        Z, Z_centered, gamma, beta, mu, sigma_sq = batch_norm_forward(parameters['W' + str(l)], parameters['b' + str(l)],
                                                                     parameters['gamma' + str(l)], parameters['beta' + str(l)], A)
        A = relu(Z)
        cache = (Z, Z_centered, gamma, beta, mu, sigma_sq, A)
        caches.append(cache)

    Z, Z_centered, gamma, beta, mu, sigma_sq = batch_norm_forward(parameters['W' + str(L)], parameters['b' + str(L)],
                                                                 parameters['gamma' + str(L)], parameters['beta' + str(L)], A)
    AL = sigmoid(Z)
    cache = (Z, Z_centered, gamma, beta, mu, sigma_sq, AL)
    caches.append(cache)

    return AL, caches

In [ ]:
def backward_propagation_with_batch_norm(AL, Y, caches):
    """
    Implements the backward propagation with batch normalization.

    Arguments:
    AL -- output of the last layer.
    Y -- true "label" vector.
    caches -- list of caches containing intermediate values.

    Returns:
    grads -- dictionary containing gradients with respect to different parameters.
    """
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)

    # Backward propagation for the last layer
    dZL = AL - Y
    grads["dW" + str(L)], grads["db" + str(L)], grads["dgamma" + str(L)], grads["dbeta" + str(L)] = \
        batch_norm_backward(dZL, caches[L - 1])
    grads["dA" + str(L - 1)] = np.dot(caches[L - 1][0], grads["dZ" + str(L)])

    # Backward propagation for hidden layers
    for l in reversed(range(1, L)):
        dA_prev_temp = np.dot(grads["dZ" + str(l + 1)], caches[l][0].T)
        grads["dA" + str(l)] = dA_prev_temp
        dZ_temp, grads["dW" + str(l)], grads["db" + str(l)], grads["dgamma" + str(l)], grads["dbeta" + str(l)] = \
            batch_norm_backward(grads["dA" + str(l)], caches[l - 1])
        grads["dA" + str(l - 1)] = np.dot(caches[l - 1][0], dZ_temp)

    return grads